In [1]:
import numpy as np

# 例如，第一道工序在机器0上，第二道工序在机器1上
chromosome = [
    [(0, 67), (1, 42), (0, 41), (1, 85), (0, 7)],  # 作业1
    [(1, 61), (0, 53), (1, 4), (0, 93)],  # 作业2
    [(1, 36), (0, 0), (1, 27), (0, 24)]  # 作业3
]

def fitness(chromosome):
    machine_schedule = {}  # 记录每个机器的使用时间
    makespan = 0

    # 遍历每个作业的每道工序
    for job_index, job in enumerate(chromosome):
        current_time = 0  # 该作业的开始时间
        for op_index, (machine, processing_time) in enumerate(job):
            if machine not in machine_schedule:
                machine_schedule[machine] = []

            # 确保没有资源冲突
            machine_busy = machine_schedule[machine]
            start_time = max(current_time, max([end for _, end in machine_busy], default=0))
            end_time = start_time + processing_time
            machine_schedule[machine].append((start_time, end_time))

            # 更新作业的开始时间
            current_time = end_time

        # 更新最大完工时间
        makespan = max(makespan, current_time)

    return makespan  # 目标是最小化完工时间

def select_population(population, fitness_values):
    # 轮盘赌选择
    total_fitness = sum(fitness_values)
    selection_probs = [fitness / total_fitness for fitness in fitness_values]
    selected = np.random.choice(population, size=len(population)//2, p=selection_probs)
    return selected

def crossover(parent1, parent2):
    # 单点交叉
    crossover_point = np.random.randint(len(parent1))
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(chromosome, mutation_rate):
    for job_index in range(len(chromosome)):
        for op_index in range(len(chromosome[job_index])):
            if np.random.rand() < mutation_rate:
                # 随机改变机器选择
                new_machine = np.random.choice([0, 1])
                chromosome[job_index][op_index] = (new_machine, chromosome[job_index][op_index][1])
    return chromosome



def initialize_population(job_length, op_pt, population_size):
    population = []
    for _ in range(population_size):
        chromosome = []
        for job in range(len(job_length)):
            job_chromosome = []
            for op in range(job_length[job]):
                machine = np.random.choice([0, 1])
                processing_time = op_pt[job * job_length[job] + op][machine]
                job_chromosome.append((machine, processing_time))
            chromosome.append(job_chromosome)
        population.append(chromosome)
    return population

def genetic_algorithm(job_length, op_pt, population_size=50, generations=100, mutation_rate=0.05):
    population = initialize_population(job_length, op_pt, population_size)
    for gen in range(generations):
        fitness_values = [fitness(individual) for individual in population]
        selected = select_population(population, fitness_values)

        next_generation = []
        for i in range(0, len(selected), 2):
            parent1, parent2 = selected[i], selected[i+1]
            child1, child2 = crossover(parent1, parent2)
            next_generation.append(mutate(child1, mutation_rate))
            next_generation.append(mutate(child2, mutation_rate))

        population = next_generation

    best_individual = min(population, key=fitness)
    return best_individual

In [2]:
import os, sys
os.environ['ON_PY']="1"
from data_utils import CaseGenerator, SD2_instance_generator
from params import configs
n_j=17
n_m=11
try:
    JobLength = np.load("job_length.npy")
    OpPT = np.load("op_pt.npy")
except:
    JobLength, OpPT, _ = SD2_instance_generator(configs, n_j=n_j, n_m=n_m, op_per_job=0)



In [3]:
%%time
schedule = genetic_algorithm(JobLength, OpPT)


ValueError: a must be 1-dimensional

In [4]:

# 打印结果
for task in schedule:
    print(task)


NameError: name 'schedule' is not defined